In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
#import plotly.express as px
#import plotly.graph_objects as go
#from plotly.subplots import make_subplots
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.metrics import Recall,AUC
from tensorflow.keras.utils import plot_model

In [21]:
#create an empty DataFrame
df = pd.DataFrame(columns=['path','label'])

#loop over fire images and label them 1
for dirname, _, filenames in os.walk("/Users/lindseyb/Documents/Fall 2021/Project/fire_dataset/fire_images"):
    for filename in filenames:
        #print(os.path.join(dirname, filename))
        df = df.append(pd.DataFrame([[os.path.join(dirname, filename),'fire']],columns=['path','label']))

#loop over non fire images and label them 0
for dirname, _, filenames in os.walk("/Users/lindseyb/Documents/Fall 2021/Project/fire_dataset/non_fire_images"):
    for filename in filenames:
        df = df.append(pd.DataFrame([[os.path.join(dirname, filename),'non_fire']],columns=['path','label']))
        #print(os.path.join(dirname, filename))

#shuffle the dataset for redistribute the labels
df = df.sample(frac=1).reset_index(drop=True)
df.head(10)


,path,label
0,/Users/lindseyb/Documents/Fall 2021/Project/fi...,fire
1,/Users/lindseyb/Documents/Fall 2021/Project/fi...,fire
2,/Users/lindseyb/Documents/Fall 2021/Project/fi...,fire
3,/Users/lindseyb/Documents/Fall 2021/Project/fi...,non_fire
4,/Users/lindseyb/Documents/Fall 2021/Project/fi...,non_fire
5,/Users/lindseyb/Documents/Fall 2021/Project/fi...,non_fire
6,/Users/lindseyb/Documents/Fall 2021/Project/fi...,fire
7,/Users/lindseyb/Documents/Fall 2021/Project/fi...,non_fire
8,/Users/lindseyb/Documents/Fall 2021/Project/fi...,fire
9,/Users/lindseyb/Documents/Fall 2021/Project/fi...,fire


In [22]:
def shaper(row):
    shape = image.load_img(row['path']).size
    row['height'] = shape[1]
    row['width'] = shape[0]
    return row
df = df.apply(shaper,axis=1)
df.head(5)

,path,label,height,width
0,/Users/lindseyb/Documents/Fall 2021/Project/fi...,fire,720,1280
1,/Users/lindseyb/Documents/Fall 2021/Project/fi...,fire,270,390
2,/Users/lindseyb/Documents/Fall 2021/Project/fi...,fire,480,640
3,/Users/lindseyb/Documents/Fall 2021/Project/fi...,non_fire,185,560
4,/Users/lindseyb/Documents/Fall 2021/Project/fi...,non_fire,805,1920


In [24]:
generator = ImageDataGenerator(
    rotation_range= 20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range = 2,
    zoom_range=0.2,
    rescale = 1/255,
    validation_split=0.2,
)

In [30]:
train_gen = generator.flow_from_dataframe(df,x_col='path',y_col='label',images_size=(256,256),class_mode='binary',subset='training')
val_gen = generator.flow_from_dataframe(df,x_col='path',y_col='label',images_size=(256,256),class_mode='binary',subset='validation')




Found 800 validated image filenames belonging to 2 classes.
Found 199 validated image filenames belonging to 2 classes.


In [ ]:
class_indices = {}
for key in train_gen.class_indices.keys():
    class_indices[train_gen.class_indices[key]] = key
    
print(class_indices)

In [34]:
model = Sequential()
model.add(Conv2D(filters=32,kernel_size = (2,2),activation='relu',input_shape = (256,256,3)))
model.add(MaxPool2D())
model.add(Conv2D(filters=64,kernel_size=(2,2),activation='relu'))
model.add(MaxPool2D())
model.add(Conv2D(filters=128,kernel_size=(2,2),activation='relu'))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dense(32,activation = 'relu'))
model.add(Dense(1,activation = 'sigmoid'))

In [35]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 255, 255, 32)      416       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 127, 127, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 126, 126, 64)      8256      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 63, 63, 64)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 62, 62, 128)       32896     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 31, 31, 128)      0

In [37]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy',Recall(),AUC()])

In [38]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau


In [39]:
early_stoppping = EarlyStopping(monitor='val_loss',patience=5,restore_best_weights=True)
reduce_lr_on_plateau = ReduceLROnPlateau(monitor='val_loss',factor=0.1,patience=5)

In [40]:
model.fit(x=train_gen,batch_size=32,epochs=15,validation_data=val_gen,callbacks=[early_stoppping,reduce_lr_on_plateau])

Epoch 1/15
25/25 [==============================] - 79s 3s/step - loss: 0.4360 - accuracy: 0.7825 - recall: 0.5431 - auc: 0.8216 - val_loss: 0.1552 - val_accuracy: 0.9698 - val_recall: 0.9149 - val_auc: 0.9956 - lr: 0.0010
Epoch 2/15
25/25 [==============================] - 94s 4s/step - loss: 0.1674 - accuracy: 0.9312 - recall: 0.8325 - auc: 0.9770 - val_loss: 0.0776 - val_accuracy: 0.9749 - val_recall: 0.9149 - val_auc: 0.9980 - lr: 0.0010
Epoch 3/15
25/25 [==============================] - 81s 3s/step - loss: 0.1232 - accuracy: 0.9563 - recall: 0.8934 - auc: 0.9873 - val_loss: 0.0648 - val_accuracy: 0.9698 - val_recall: 1.0000 - val_auc: 0.9988 - lr: 0.0010
Epoch 4/15
25/25 [==============================] - 79s 3s/step - loss: 0.1300 - accuracy: 0.9425 - recall: 0.8985 - auc: 0.9849 - val_loss: 0.1204 - val_accuracy: 0.9497 - val_recall: 0.9574 - val_auc: 0.9920 - lr: 0.0010
Epoch 5/15
25/25 [==============================] - 79s 3s/step - loss: 0.0979 - accuracy: 0.9625 - recall: 

In [41]:
eval_list = model.evaluate(val_gen,return_dict=True)
for metric in eval_list.keys():
    print(metric+f": {eval_list[metric]:.2f}")

7/7 [==============================] - 10s 1s/step - loss: 0.0614 - accuracy: 0.9749 - recall: 1.0000 - auc: 0.9990
loss: 0.06
accuracy: 0.97
recall: 1.00
auc: 1.00


In [42]:
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout

In [43]:
xception = Xception(include_top = False,input_shape = (256,256,3))
input_to_model = xception.input
#turn off training
xception.trainable = False

x = Flatten()(xception.output)
x = Dense(64,activation = 'relu')(x)
output_to_model = Dense(1,activation = 'sigmoid')(x)
model2 = Model(inputs = input_to_model,outputs = output_to_model)

83697664/83683744 [==============================] - 9s 0us/step


In [44]:
model2.compile(optimizer = 'adam',loss = 'binary_crossentropy',metrics = ['accuracy',Recall(),AUC()])

In [45]:
history2 = model2.fit(x = train_gen,batch_size=32,epochs=15,callbacks = [early_stoppping,reduce_lr_on_plateau],validation_data = val_gen)

Epoch 1/15
25/25 [==============================] - 188s 7s/step - loss: 1.0995 - accuracy: 0.8662 - recall_1: 0.7716 - auc_1: 0.8949 - val_loss: 0.3976 - val_accuracy: 0.9296 - val_recall_1: 0.7447 - val_auc_1: 0.9323 - lr: 0.0010
Epoch 2/15
25/25 [==============================] - 195s 8s/step - loss: 0.2285 - accuracy: 0.9550 - recall_1: 0.8883 - auc_1: 0.9758 - val_loss: 0.1558 - val_accuracy: 0.9548 - val_recall_1: 0.8298 - val_auc_1: 0.9656 - lr: 0.0010
Epoch 3/15
25/25 [==============================] - 183s 7s/step - loss: 0.1318 - accuracy: 0.9725 - recall_1: 0.9492 - auc_1: 0.9882 - val_loss: 0.0795 - val_accuracy: 0.9698 - val_recall_1: 0.8723 - val_auc_1: 0.9987 - lr: 0.0010
Epoch 4/15
25/25 [==============================] - 2677s 111s/step - loss: 0.1102 - accuracy: 0.9737 - recall_1: 0.9289 - auc_1: 0.9873 - val_loss: 0.0390 - val_accuracy: 0.9849 - val_recall_1: 0.9787 - val_auc_1: 0.9986 - lr: 0.0010
Epoch 5/15
25/25 [==============================] - 924s 34s/step - l

KeyboardInterrupt: 